In [ ]:
!pip install pyspellchecker
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tqdm.notebook import tqdm
tqdm.pandas()

import re
from spellchecker import SpellChecker

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load data

In [ ]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

# Clean data

remove username, urls, etc

In [ ]:
def clean(text):
    text = text.fillna("fillna").str.lower()
    text = text.map(lambda x: re.sub('\\n',' ',str(x)))
    text = text.map(lambda x: re.sub("\[\[User.*",'',str(x)))
    text = text.map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
    text = text.map(lambda x: re.sub("\(http://.*?\s\(http://.*\)",'',str(x)))
    return text

In [ ]:
train1["comment_text"] = clean(train1["comment_text"])
train2["comment_text"] = clean(train2["comment_text"])
valid["comment_text"] = clean(valid["comment_text"])
test["content"] = clean(test["content"])

Remove entries with more misspells

In [ ]:
spell = SpellChecker()

In [ ]:
%%time 
# Counting the number of spelling errors
train1['mispell_count'] = train1['comment_text'].progress_apply(lambda x: len(spell.unknown(x.split())))
train2['mispell_count'] = train2['comment_text'].progress_apply(lambda x: len(spell.unknown(x.split())))

In [ ]:
train1[train1['mispell_count'] < 100]['mispell_count'].hist(bins=100)

In [ ]:
train2[train2['mispell_count'] < 100]['mispell_count'].hist(bins=100)

Check how many toxic comments have more than 20 mispells (Labels are already imbalanced; We do not want to lose them)

In [ ]:
sns.countplot(train1[train1['mispell_count'] >= 20].toxic)

In [ ]:
sns.countplot(train2[train2['mispell_count'] >= 20].toxic)

Remove non-toxic comments with more than 20 misspells

In [ ]:
train1 = train1.query('toxic==0 & mispell_count < 20 | toxic==1')
train2 = train2.query('toxic==0 & mispell_count < 20 | toxic==1')

**Subsample the data labeled 0 to balance the labeling**

In [ ]:
# Examine number of data labeled 1 or 0
print('train2 has %d entries labeled 1' % train2[['comment_text', 'toxic']].query('toxic==1').shape[0])
print('train1 has %d entries labeled 1' % train1[['comment_text', 'toxic']].query('toxic==1').shape[0])
print('train2 has %d entries labeled 0' % train2[['comment_text', 'toxic']].query('toxic==0').shape[0])
print('train1 has %d entries labeled 0' % train1[['comment_text', 'toxic']].query('toxic==0').shape[0])

In [ ]:
# Combine train1 with a subset of train2 
# (decided to subsample 100,000 samples from train2 labeld 0)
train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])

# Encode data

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [ ]:
# First load the real tokenizer
MODEL = 'jplu/tf-xlm-roberta-large'
# Configuration
MAX_LEN = 192
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
%%time 

x_train = regular_encode(train.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen=MAX_LEN)

In [ ]:
y_train = train.toxic.values
y_valid = valid.toxic.values

# TPU configs

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

# Build dataset

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(len(train))
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

# Build model

In [ ]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', 
                  metrics=[
                          keras.metrics.TruePositives(name='tp'),
                          keras.metrics.FalsePositives(name='fp'),
                          keras.metrics.TrueNegatives(name='tn'),
                          keras.metrics.FalseNegatives(name='fn'), 
                          keras.metrics.BinaryAccuracy(name='accuracy'),
                          keras.metrics.Precision(name='precision'),
                          keras.metrics.Recall(name='recall'),
                          keras.metrics.AUC(name='auc')
                  ])

    return model

In [ ]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

# Train model

In [ ]:
matplotlib.rcParams['figure.figsize'] = (12, 10)

In [ ]:
def plot_metrics(history, val=True):
    metrics =  ['accuracy', 'auc', 'precision', 'recall']
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2, 2, n+1)
        plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
        if val:
            plt.plot(history.epoch, history.history['val_' + metric],
                     color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        plt.ylim([0, 1])

        plt.legend()

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

In [ ]:
plot_metrics(train_history)

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS
)

In [ ]:
plot_metrics(train_history_2, val=False)

# Predict

In [ ]:
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)